In [50]:
import tensorflow as tf
import numpy as np

In [106]:
# Load training and eval data
((train_data, train_labels), (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [108]:
train_data.shape

(60000, 28, 28)

In [109]:
train_labels.shape

(60000,)

### neural network architecture

dimensions (filter_height x filter_width x filter_number)

* layer_type (filter details)
* conv1 (10 x 10 x 10)->
* maxPool1 (5 x 5) -> 
* conv2 (8 x 8 x 10)->
* fc 
* softmax

In [132]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical

In [101]:
model = Sequential()
model.fit()

In [133]:
y_train = to_categorical(train_labels)

In [134]:
y_train.shape

(60000, 10)

In [129]:
X_train = train_data.reshape(60000,28,28, 1)

In [147]:



model = Sequential()

model.add(Conv2D(256, (10, 10), input_shape = (28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(5, 5)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))

model.add(Dense(10))

model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['categorical_accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=3, validation_split=0.3, use_multiprocessing = True)

Train on 42000 samples, validate on 18000 samples
Epoch 1/3
25376/42000 [=================>............] - ETA: 21s - loss: 0.2023 - categorical_accuracy: 0.9370

KeyboardInterrupt: 

In [145]:
model.history.history

{'loss': [0.15562202490529134, 0.055974973812477576, 0.04027803984423545],
 'categorical_accuracy': [0.95088094, 0.9818571, 0.98735714],
 'val_loss': [0.08177818952335252, 0.05847911647365739, 0.04885806441514028],
 'val_categorical_accuracy': [0.9723333, 0.9826111, 0.98483336]}

In [79]:
from CNN_helper_functions import *

In [ ]:
get_conv_output_dims

In [95]:
def cnn_model(features, labels, mode = 'TRAIN'):
    
    
    X = tf.placeholder(dtype=tf.float32, shape=[None,784])
    
    input_layer = tf.reshape(X, [-1, 28, 28, 1])
    
    conv_1 = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=[10, 10],
    padding="same",
    activation=tf.nn.relu)(input_layer)
    
    pool_1 = tf.layers.max_pooling2d(inputs=conv_1, pool_size=(5,5), strides=1, )
    
    conv_2 = tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=[8, 8],
    padding="same",
    activation=tf.nn.relu)(pool_1)
    
    
    conv_2_flat = tf.reshape(conv_2, [?????])
    
    logits = tf.layers.dense(inputs=conv_2_flat, units=10)

    
    tf.argmax(input=logits, axis=1)
    
    tf.nn.softmax(logits, name="softmax_tensor")
    
    
    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
        }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    #loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
          loss=loss,
           global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
    }
    return tf.estimator.EstimatorSpec(

        mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

    

In [96]:
# Create the Estimator
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c401be400>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [97]:
# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}

logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)


In [98]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

In [99]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)

INFO:tensorflow:Calling model_fn.


ValueError: Dimension size must be evenly divisible by 18432 but is 18430 for 'gradients/Reshape_1_grad/Reshape' (op: 'Reshape') with input shapes: [10,1843], [4] and with input tensors computed as partial shapes: input[1] = [?,24,24,32].